# Mulitagent RAG

In [87]:
# Download a file
# import wget

file_path = "my_document.pdf"
# doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"

# wget.download(doc_url, out=file_path)

In [88]:
# Qdrant similarity search
from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.embeddings import HuggingFaceHubEmbeddings

import os
import dotenv

dotenv.load_dotenv()
# embedder
EMBEDDINGS_URL = os.getenv("EMBEDDINGS_URL")
EMBEDDINGS_KEY = os.getenv("EMBEDDINGS_KEY")

print("EMBEDDINGS_URL", EMBEDDINGS_URL)
print("EMBEDDINGS_KEY", EMBEDDINGS_KEY)
# model_name = "BAAI/bge-large-en-v1.5"
# model_kwargs = {"device": "cpu"}
# encode_kwargs = {"normalize_embeddings": True}
# embedder = HuggingFaceBgeEmbeddings(
#     model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
# )
# embedder = OpenAIEmbeddings(openai_api_key=EMBEDDINGS_KEY, model="BAAI/bge-m3", base_url=EMBEDDINGS_URL)
embedder = HuggingFaceHubEmbeddings(model=EMBEDDINGS_URL, huggingfacehub_api_token=EMBEDDINGS_KEY)


client = QdrantClient(
    url="http://localhost:6333", api_key="multivac-FQ1cWX4DpshdhkXY2m"
)
db = Qdrant(client=client, collection_name="decision-adlc", embeddings=embedder)
colls = client.get_collections()
collection_names = [x["name"] for x in colls.dict()["collections"]]
question = "Un contrat de CDI doit-il être forcément écrit ?"

docs = []

for coll in collection_names:
    db = Qdrant(client=client, collection_name=coll, embeddings=embedder)
    docs_ = db.similarity_search_with_score(question.lower(), k=5)
    docs_ = [(*doc, coll) for doc in docs_]
    docs = docs + docs_


docs = sorted(docs, key=lambda x: x[1], reverse=True)

print("docs", docs)

EMBEDDINGS_URL http://albert.gpu.005.etalab.gouv.fr:8001/v1
EMBEDDINGS_KEY EMPTY
docs [(Document(metadata={'date': '2023-06-12', 'keywords': "['contrat', 'écrit', 'temps', 'écrits', 'indéterminée', 'partiel', 'msa', 'contrats', 'durée', 'complet']", 'potential_questions': '[ "Quels sont les contrats de travail qui ne doivent pas être écrits ?", "Quels sont les documents que l\'employeur doit remettre au salarié en cas de contrat verbal ?" ]', 'pubId': 'article100976', 'title': 'Contrat de travail\xa0: les principales caractéristiques', 'url': 'https://travail-emploi.gouv.fr/droit-du-travail/la-vie-du-contrat-de-travail/article/contrat-de-travail-les-principales-caracteristiques', '_id': '85e738d9-552f-4f8a-8d00-e935f93d8a57', '_collection_name': 'fiches-travail'}, page_content='potential_questions : [ "Quels sont les contrats de travail qui ne doivent pas être écrits ?", "Quels sont les documents que l\'employeur doit remettre au salarié en cas de contrat verbal ?" ] \n keywords : [\'c

In [89]:
# Request configuration
import requests

base_url = "http://localhost:8080/v1"
api_key = "albert" # enter any value if no api_key is setup in config.ini file

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}  # skip headers if no api_key is setup in config.ini file

In [90]:
# Upload a file
import os

collection = "leo"
model = "BAAI/bge-m3"
params = {"collection": collection, "model": model} 

files = {'files': (os.path.basename(file_path), open(file_path, 'rb'), "application/pdf")}
response = session.post(f"{base_url}/files", params=params , files=files)

response.json()

{'object': 'list',
 'data': [{'object': 'upload',
   'id': 'ab194eec-4ff8-4919-a076-2ec64da7b9b2',
   'filename': 'my_document.pdf',
   'status': 'success'}]}

In [91]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{collection}")

response.json()

{'object': 'list',
 'data': [{'object': 'file',
   'id': '060bcaab-0d56-4b99-89b8-8d1df5b038cc',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1721385337},
  {'object': 'file',
   'id': '093f8336-75ce-4c5c-afb2-98cf63c094da',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1721206658},
  {'object': 'file',
   'id': '096d32e4-6f88-41be-85b4-ba504887108b',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1721981635},
  {'object': 'file',
   'id': '0adecae4-527b-4b61-8912-894fc54834a6',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1721981668},
  {'object': 'file',
   'id': '0ce3ca59-d5b3-4f88-be56-39bc26b991e5',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1721386967},
  {'object': 'file',
   'id': '0e6feb34-ab04-49f3-9acb-7bc29cbb3d01',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1721729234},
  {'object': 'file',
   'id': '1b356b99-48ec-4e54

In [92]:
file_id = response.json()["data"][0]["id"]

In [93]:
# Get tools
response = session.get(f"{base_url}/tools")

response.json()

{'object': 'list',
 'data': [{'id': 'BaseRAG',
   'description': 'Base RAG, basic retrival augmented generation.\n\n    Args:\n        embeddings_model (str): OpenAI embeddings model\n        collection (Optional[List[str]], optional): List of collections to search in. Defaults to None (all collections).\n        file_ids (Optional[List[str]], optional): List of file IDs in the selected collections (after upload files). Defaults to None (all files are selected).\n        k (int, optional): Top K per collection (max: 6). Defaults to 4.\n        prompt_template (Optional[str], optional): Prompt template. Defaults to DEFAULT_PROMPT_TEMPLATE.',
   'object': 'tool'},
  {'id': 'UseFiles',
   'description': 'Fill your prompt with file contents. Your prompt must contain "{files}" placeholder.\n\n    Args:\n        collection (str): Collection name.\n        file_ids (List[str]): List of file ids in the selected collection.',
   'object': 'tool'},
  {'id': 'MultiAgents',
   'description': 'Mult

In [94]:
# Display tools parameters
for tool in response.json()["data"]:
    if tool["id"] == "MultiAgents":
        print(tool["description"])

MultiAgents, multiple agents for RAG: Recursive Document Retrieval & Web Search.

    Args:
        clients (dict): Dictionary containing initialized clients.


In [95]:
# OpenAI client configuration
from openai import OpenAI

client = OpenAI(base_url=base_url, api_key=api_key)

In [96]:
models = session.get(f"{base_url}/models")
print("models: ", models.json()["data"])

models:  [{'id': 'meta-llama/Meta-Llama-3-8B-Instruct', 'created': 1721984824, 'object': 'model', 'owned_by': 'vllm', 'root': 'meta-llama/Meta-Llama-3-8B-Instruct', 'parent': None, 'permission': [{'id': 'modelperm-cf949144de754d1894be622d3639fe81', 'object': 'model_permission', 'created': 1721984824, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}]}, {'id': 'gradientai/Llama-3-8B-Instruct-262k', 'created': 1721984824, 'object': 'model', 'owned_by': 'vllm', 'root': 'gradientai/Llama-3-8B-Instruct-262k', 'parent': None, 'permission': [{'id': 'modelperm-3dff31dc92f044b39c084c5ba7342cfe', 'object': 'model_permission', 'created': 1721984824, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': N

In [97]:
# Chat completions
user = "leo"
chat_id = "0e64d044-1c1f-4c49-832a-2b426efe4fa8"

data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "Peut-on avoir des jours de congés pour un mariage ?"}],
    "stream": False,
    "n": 1,
    "user": user,
    "tools": [
        {
            "function": {
                "name": "MultiAgents",
                "parameters": {
                    "embeddings_model": model,
                    "chat_id": chat_id,
                },
            },
            "type": "function",
        }
    ],
}

response = client.chat.completions.create(**data)
print(response.choices[0].message.content)

Oui, selon l'extrait 'Les congés pour événements familiaux et le congé de deuil', le salarié peut bénéficier d'un congé pour un mariage. Il peut prendre ce congé au moment de l'événement, mais pas nécessairement le jour du mariage, mais dans la période entourant cet événement. 🎉


In [98]:
# Chat completions using Web 
# Instead of using the Qdrant knowledge base, we can use the Web tool to get completions by adding web: in the message content

user = "leo"
chat_id = "0e64d044-1c1f-4c49-832a-2b426efe4fa8"

data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "web: Peut-on avoir des jours de congés pour un mariage ?"}],
    "stream": False,
    "n": 1,
    "user": user,
    "tools": [
        {
            "function": {
                "name": "MultiAgents",
                "parameters": {
                    "embeddings_model": model,
                    "chat_id": chat_id,
                },
            },
            "type": "function",
        }
    ],
}

response = client.chat.completions.create(**data)
print(response.choices[0].message.content)

Oui, selon le Code du travail, le salarié peut bénéficier de 4 jours de congés pour son propre mariage ou pour la conclusion d'un PACS. 🎉


In [99]:
# View chat history
chat_id = response.id

response = session.get(url=f"{base_url}/chat/history/{user}/{chat_id}")
print(response.json()["messages"])

[{'content': 'web: Peut-on avoir des jours de congés pour un mariage ?', 'role': 'user'}, {'role': 'assistant', 'content': "Oui, selon le Code du travail, le salarié peut bénéficier de 4 jours de congés pour son propre mariage ou pour la conclusion d'un PACS. 🎉"}]
